In [84]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from sklearn import cross_validation
from sklearn import tree
import warnings
warnings.filterwarnings('ignore')

In [85]:
#traing data set
df=pd.read_csv('file:///C:/Users/Rishab/PycharmProjects/Machine_Learning_project/Music_recomandation_sys/Untitled Folder/train.csv')
print('train_data',df.shape)

train_data (7377418, 6)


In [86]:
#merge the song and member data_sets
songs=pd.read_csv('file:///D:/data/ML Data/final_codes/songs.csv')

#df=pd.concat([df,songs])
df = pd.merge(df, songs, on='song_id', how='left')


del songs
#print(df.shape)
members=pd.read_csv('file:///D:/data/ML Data/final_codes/members.csv')
df = pd.merge(df, members, on='msno', how='left')


#df=pd.concat([df,members])
del members
print(df.shape)
#print(df.head())


(7377418, 18)


In [87]:
#removing null values
df.isnull().sum()/df.isnull().count()*100

msno                       0.000000
song_id                    0.000000
source_system_tab          0.336825
source_screen_name         5.622618
source_type                0.291959
target                     0.000000
song_length                0.001545
genre_ids                  1.605643
artist_name                0.001545
composer                  22.713990
lyricist                  43.088219
language                   0.002033
city                       0.000000
bd                         0.000000
gender                    40.142486
registered_via             0.000000
registration_init_time     0.000000
expiration_date            0.000000
dtype: float64

In [88]:
#removing the not avialable values from df

for i in df.select_dtypes(include=['object']).columns:
    df[i][df[i].isnull()]='unknown'
df=df.fillna(value=0)


In [89]:
#creating a column of Registration initial date using mm-yy-dd

df.registration_init_time=pd.to_datetime(df.registration_init_time, format='%Y%m%d',errors='ignore')

df['registration_init_time_year']=df['registration_init_time'].dt.year
df['registration_init_time_month']=df['registration_init_time'].dt.month
df['registration_init_time_day']=df['registration_init_time'].dt.day

#print(df.head())
print(df.shape)

(7377418, 21)


In [90]:
#creating a column expiration of using yy-mm-dd
df.expiration_date=pd.to_datetime(df.expiration_date, format='%Y%m%d',errors='ignore')

df['expiration_date_year']=df['expiration_date'].dt.year
df['expiration_date_month']=df['registration_init_time'].dt.month
df['expiration_date_day']=df['registration_init_time'].dt.day

#print(df.head())
print(df.shape)

(7377418, 24)


In [91]:
# convertion the this two column value into catagorial value

df['registration_init_time']=df['registration_init_time'].astype('category')

df['expiration_date']=df['expiration_date'].astype('category')

#print(df.head())
print(df.shape)

(7377418, 24)


In [92]:
# converting into categorcial values
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype('category')
     
# encoding of categorical values
for col in df.select_dtypes(include=['category']).columns:
    df[col] = df[col].cat.codes
df = df.drop(['expiration_date', 'lyricist'], 1)     

In [31]:
print(df.shape)

(7377418, 22)


In [93]:
from sklearn.decomposition import PCA
#principle component analysis

pca=PCA(n_components=2)

t=pca.fit(df[df.columns[df.columns!='target']])

principlecomponent=pca.fit_transform(df)
principal=pd.DataFrame(data=principlecomponent,columns=['pc1','pc2'])
print(principal)
finalDF=pd.concat([principal,df[['target']]],axis=1)

                   pc1            pc2
0        103927.782413  -41828.100225
1        -43431.958313   40456.680224
2         58365.577785  -20895.564491
3        155735.021548    7813.688758
4        145020.658382  -61185.107750
5        149241.653547     393.157961
6        -10787.663556  -15416.849209
7        -43112.327229  -63170.283963
8        170656.255042   31487.814727
9         85750.305483   10989.022956
10      -141881.583393  -20102.257157
11       121201.879830  -21417.026650
12      -147063.801731   37564.938944
13         2012.327035    8957.131054
14        62074.729007  -36205.328373
15       132249.022880   -7181.681602
16       -34650.092443    4721.623371
17      -141982.225027   16955.176134
18       142066.729127   18114.764855
19       111464.558120   -4897.592996
20       -11708.341747  -54726.401072
21        72155.449542    2185.628143
22      -100862.931922   14359.376607
23       -70594.500333  -61971.782661
24      -107410.256179   47224.781553
25       -15

In [94]:
#creating decision tree classifier
model = tree.DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False)
t=model.fit(df[df.columns[df.columns != 'target']], df.target)
print(t)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')


In [98]:
df_plot = pd.DataFrame({'features': df.columns[df.columns != 'target'],
                        'importances': model.feature_importances_})


print(df_plot)


ValueError: arrays must all be same length

In [96]:
df_plot=df_plot.sort_values('importances',ascending=False)
print(df_plot)

                        features  importances
5                    song_length     0.129571
1                        song_id     0.127781
0                           msno     0.107975
7                    artist_name     0.098236
14        registration_init_time     0.083239
8                       composer     0.081036
4                    source_type     0.069978
11                            bd     0.038430
20           expiration_date_day     0.036736
17    registration_init_time_day     0.036553
6                      genre_ids     0.030524
10                          city     0.030280
16  registration_init_time_month     0.021334
19         expiration_date_month     0.021054
9                       language     0.015260
3             source_screen_name     0.014851
2              source_system_tab     0.014396
13                registered_via     0.013856
12                        gender     0.010926
15   registration_init_time_year     0.009573
18          expiration_date_year  

In [97]:
df = df.drop(df_plot.features[df_plot.importances < 0.04].tolist(), 1)


In [74]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)
y = df['target'].values
model = tree.DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False)
for train_indices,val_indices in kf.split(df) :
    model.fit(df.drop(['target'],axis=1).loc[train_indices,:],df.loc[train_indices,:],df.loc[train_indices,'target'])

KeyError: 'target'